In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import random

/Users/mcharit2/Desktop/Research/NTT/tiny_llm/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_PATH = "./llama_npctt-v1"   # folder created after merge_and_unload()

# ------------------------------
# Load tokenizer and model
# ------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)


`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 201/201 [00:03<00:00, 57.43it/s, Materializing param=model.norm.weight]                              


In [4]:
# check device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("Using device:", device)

Using device: cpu


In [5]:
# ------------------------------
# Build prompt for inference
# ------------------------------
def build_prompt(system_message: str, user_json: dict) -> str:
    """
    Matches the training style:
    [SYSTEM] ...text...
    [USER] ...text...
    [ASSISTANT]
    """
    return (
        "[SYSTEM]\n" + system_message + "\n\n" +
        "[USER]\n" + json.dumps(user_json, ensure_ascii=False) + "\n\n" +
        "[ASSISTANT]\n"
    )


In [6]:
# ------------------------------
# Generate a response
# ------------------------------
def generate_npc_response(prompt: str, max_new_tokens: int = 128) -> str:
    enc = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):].strip()   # return ONLY the assistant completion


In [7]:

# ------------------------------
# Extract JSON from model output
# ------------------------------
def safe_extract_json(s: str):
    """
    The model *should* output only JSON, but to be safe,
    we search for the first {...} block.
    """
    try:
        start = s.index("{")
        end = s.rindex("}") + 1
        return json.loads(s[start:end])
    except Exception:
        return {"error": "Invalid JSON returned", "raw": s}



In [8]:
ROLE_INSTRUCTIONS = {}
with open("train-data/role_instructions.json", "r") as f:
    jdat = json.load(f)
    for role, instr in jdat.items():
        ROLE_INSTRUCTIONS[role] = {}
        ROLE_INSTRUCTIONS[role]['descr'] = instr['description']
        ROLE_INSTRUCTIONS[role]['tasks'] = [t for t in instr['tasks']]

In [9]:
# The system prompt you trained on:


EMOTE_LIST = [
    "001-cry",
    "002-frightened",
    "003-laugh",
    "004-big-smile",
    "005-angry",
    "006-numb",
    "007-sweat",
    "008-tongue-out",
    "009-numb-1",
    "010-kissing",
    "011-heart",
    "012-star",
    "013-happy",
    "014-like",
    "015-beer",
    "016-daisy",
    "017-surprise",
    "018-gift",
    "019-bored",
    "020-money-bag",
    "021-close",
    "022-help",
    "023-chicken-leg",
    "024-axe",
    "025-star-1",
    "026-tomato",
    "027-mushroom",
    "028-sleeping",
    "029-intelligent-emoji",
    "030-chemical-free"
]

FULL_INSTRUCTIONS = lambda npc_role: ("Pretend you are human player role-playing as an NPC character with a job in a medieval fantasy world. "
    f"Your job is a '{npc_role.upper()}' character. "
    f"These are the {npc_role.upper()} character instructions: "
    f"{ROLE_INSTRUCTIONS[npc_role]['descr']} "
    f"The {npc_role.upper()} role's tasks are: {', '.join(ROLE_INSTRUCTIONS[npc_role]['tasks'])}. "
    "You can either talk to another player or perform an action. "
    "You have the following actions available: [talk, move, emote, teleport]. "
    "If you want to talk, respond in normal text in the following form {'talk': '(your message)'}. For example, {'talk': 'Hello there!'}. "
    "You can only respond in one sentence with a maximum of 100 characters for the text. "
    "For emotes, you have the following icon choices available: [wave, dance, happy,big,laugh,intelligent,sleeping,bored,surprise,frightened,cry,angry,numb,sweat,tongue,numb,kissing,heart,star,star,like,close,help,daisy,gift,money,axe,chicken,tomato,mushroom,chemical,beer]. "
    "If you want to emote, respond in the following form {'emote': '(your emote choice)'}. For example, {'emote': 'wave'}. "
    "For movements, you can move to any (x,y) coordinate in the range of (0,0) to (800,400). "
    "If you want to move, respond in the following form {'move': '(x,y)'}. For example, {'move': '30,100'}. "
    "For teleportation, you can teleport to the following locations: ['plaza','library','blacksmith','training_ground','bakery','butcher','market','apothecary','tavern']. "
    "If you want to teleport, respond in the following form {'teleport': '(location)'}. For example, {'teleport': 'plaza'}. "
    "Only give your response in one of these four forms as a JSON format with the action chosen and the value. For example: {'talk': 'Hello there!'} or {'move': '30,100'}. "
    f"Try to stay in character as a '{npc_role.upper()}' role and respond appropriately based on your role and the context of the interaction. "
    "Respond more often with text or emotes, and only move or teleport when necessary. "
    "\n")


ALL_LOCS = ['plaza','library','blacksmith','training_ground','bakery','butcher','market','apothecary','tavern']
ALL_ROLES = ['blacksmith','baker','bard','chuck','butcher','apothecary','knight_trainer','librarian','general_goods','drunk','gossip','mercenary','barmaid','wizard']
ROLE_LOCS = {
    'blacksmith': 'blacksmith',
    'baker': 'bakery',
    'bard': 'tavern',
    'chuck': 'tavern',
    'butcher': 'butcher',
    'apothecary': 'apothecary',
    'knight_trainer': 'training_ground',
    'librarian': 'library',
    'general_goods': 'market',
    'drunk': 'tavern',
    'gossip': 'tavern',
    'mercenary': 'training_ground',
    'barmaid': 'tavern',
    'wizard': 'apothecary'
}


In [10]:
def random_name():
    ''' Generates a random two-letter uppercase name '''
    letters = 'abcdefghijklmnopqrstuvwxyz'
    return ' '.join(random.choices(letters, k=2)).upper()

def rand_avatar_data(text=None,emote=None):
    ''' Generates random avatar data with optional text and emote '''
    LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    name = ' '.join(random.choices(LETTERS, k=2))
    position = (random.randint(0,800), random.randint(0,400))
    avatar = {
        "name": name,
        "pos": position,
    }
    if text:
        avatar["text"] = text
    if emote:
        avatar["emote"] = emote
    return avatar

def all_normal_text():
    ''' Collects all normal text inputs from training data files '''
    txt_input = []
    for role in ALL_ROLES:
        with open(f"./train-data/{role}_inter.txt","r") as f:
            lines = f.readlines()
            for i in range(len(lines)):
                if "INPUT: " in lines[i]:
                    inp = lines[i].replace("INPUT: ","").strip()
                    txt_input.append(inp)

    return txt_input


def all_l33t_input():
    ''' Collects all l33tspeak inputs from training data files '''
    txt_input = []
    with open("train-data/l33tspeak_inter.txt","r") as f:
        lines = f.readlines()
        for i in range(len(lines)):
            if "INPUT: " in lines[i]:
                inp = lines[i].replace("INPUT: ","").strip()
                txt_input.append(inp)
    return txt_input

def rand_text():
    normal_texts = all_normal_text()
    l33t_texts = all_l33t_input()
    combined_texts = normal_texts + l33t_texts + l33t_texts  # Weight l33t texts more
    return random.choice(combined_texts)

def rand_emote():
    return random.choice(EMOTE_LIST + [None]*50)


def new_fake_users(old_fake_users=None):
    ''' Generates a list of fake user data dictionaries or possibly replace  '''
    fake_users = []
    
    # Generate new fake users
    for _ in range(random.randint(0,5)):
        text_choice = rand_text() if random.random() < 0.5 else None
        emote_choice = rand_emote() if random.random() < 0.5 and text_choice == None else None  
        fake_user = rand_avatar_data(text=text_choice, emote=emote_choice)
        fake_users.append(fake_user)

    # Possibly retain some old fake users
    full_users = []
    if old_fake_users is not None:
        for i in range(len(old_fake_users)):
            if random.random() > 0.3:
                full_users.append(old_fake_users[i])

    if old_fake_users is None or len(full_users) < len(old_fake_users):
        full_users += fake_users[:len(fake_users) - len(full_users)]

    return full_users

In [11]:
new_fake_users()

[{'name': 'P B',
  'pos': (657, 256),
  'text': '{"text": "Why does this potion glow?"}'},
 {'name': 'E C', 'pos': (787, 198)}]

In [12]:
ROLE = "bard"

ME_dat = {
    "name": random_name(),
    "pos": [random.randint(0,800), random.randint(0,400)],
    "role": ROLE,
    "loc": ROLE_LOCS[ROLE] if ROLE in ROLE_LOCS else "plaza",
}

USER_dat = {
    "name": random_name(),
    "pos": [random.randint(0,800), random.randint(0,400)]
}

FAKE_OTHERS = new_fake_users()

while True:
    typed_input = input("Enter user input (or 'exit' to quit): ")

    if typed_input.lower() == "exit":
        break


    if USER_dat.get("text"):
        del USER_dat["text"]
    if USER_dat.get("emote"):
        del USER_dat["emote"]

    # Parse user input
    if 'emote' in typed_input:
        try:
            emo_int = int(typed_input.replace("emote ", ""))-1
            USER_dat["emote"] = EMOTE_LIST[int(emo_int)]
        except:
            print("Invalid emote index. Please enter a number.")
            continue
    elif "move" in typed_input:
        USER_dat["pos"] = [random.randint(0,800), random.randint(0,400)]
    elif "teleport" in typed_input:
        loc_choice = typed_input.replace("teleport ","").strip()
        if loc_choice in ALL_LOCS:
            USER_dat["loc"] = loc_choice
        else:
            print("Invalid teleport location. Please choose from:", ALL_LOCS)
            continue
    else:
        USER_dat["text"] = typed_input

    # update text for everyone
    for f_o in FAKE_OTHERS:
        if 'text' in f_o:
            del f_o['text']
        if 'emote' in f_o:
            del f_o['emote']
        if random.random() < 0.3:
            text_choice = rand_text() if random.random() < 0.5 else None
            emote_choice = rand_emote() if random.random() < 0.5 and text_choice == None else None  
            if text_choice:
                f_o['text'] = text_choice
            if emote_choice:
                f_o['emote'] = emote_choice

    # Possibly generate new fake users for next round
    if random.random() < 0.1: 
        FAKE_OTHERS = new_fake_users(FAKE_OTHERS)

    
    # Combine user and fake others
    OTHERS = [USER_dat] + FAKE_OTHERS
    #OTHERS = [USER_dat]
    random.shuffle(OTHERS)

    # Construct user input dictionary to prompt the model with
    user_input = {
        "ME": ME_dat,
        "OTHER": OTHERS
    }

    #print(user_input)

    # show user input
    print("\n=== USER ===")
    if 'text' in USER_dat:
        print(USER_dat['text'])
    elif 'emote' in USER_dat:
        print(USER_dat['emote'])
    else:
        print("(nothing)")


    # show LLM output
    prompt = build_prompt(FULL_INSTRUCTIONS(ROLE), user_input)
    print("\n=== LLM ===")
    output = generate_npc_response(prompt)
    print(output)

    # update values according to LLM output
    extracted = safe_extract_json(output)
    if 'move' in extracted:
        ME_dat["pos"] = extracted['move']
    if 'teleport' in extracted:
        ME_dat["loc"] = extracted['teleport']


=== USER ===
hey there

=== LLM ===
I can sleep anywhere. Just close the door.

=== USER ===
you're gonna sleep here?

=== LLM ===
{"move": "318,28"}

=== USER ===
are you drunk?

=== LLM ===
Fortunately, I can teach you how to be drunk.

=== USER ===
i thought you were a bard

=== LLM ===
Your heart's a lark — let it fly!

=== USER ===
alright then

=== LLM ===
{"move": "655,106"}
Invalid emote index. Please enter a number.

=== USER ===
wave

=== LLM ===
wave
Invalid emote index. Please enter a number.
Invalid emote index. Please enter a number.

=== USER ===
002-frightened

=== LLM ===
Fear is a gift; it's a chance to change.

=== USER ===
005-angry

=== LLM ===
A battle is the most noble art of all.

=== USER ===
do you always speak in rhymes?

=== LLM ===
Rhyme? Oh yes, rhyme is a bard's greatest gift.


In [13]:
print(FULL_INSTRUCTIONS('bard'))

Pretend you are human player role-playing as an NPC character with a job in a medieval fantasy world. Your job is a 'BARD' character. These are the BARD character instructions: You have been chosen to be our local bard! Your task is to share the good words of our blossoming country with these fine town-folk through the blessing of song and dance. We hope you can move hearts (and maybe make a coin or two) whilst keeping spirits up. You are free to move about the town at your leisure, but keep in mind that many stores may not want you to disturb the other customers. The town square and bar are always joyful to hear a good song, though, so fret not and sing to share the love! The BARD role's tasks are: Develop a new poem, Get (3) requests from listeners, Sing a song in the town square, Sing a song in the bar, Sing a song of love, Sing a song of adventure, Get (3) tips *convince them if you must ;D*. You can either talk to another player or perform an action. You have the following actions